# Scraping posts from r/conspiracy

Extraction of all the posts resent in r/conspiracy, with title, content and url.
First of all we need to import praw and access reddit:

In [4]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "YUuwWt8-PMY44mNE7QJ0xw", client_secret= "TPohIg8dyysup5fmaQ96MhIQmunLOA", user_agent= "epds_son")

We can now access the subreddit, extract all the posts url in it and check which other subreddit reposted that posts and how much:

In [ ]:
import pandas as pd
post_list=list()
subreddit_list = list()
conspiracy_dict=dict()

for i in reddit.subreddit("conspiracy").top(limit=5000): #recupera i 5000 post top di conspiracy
    post_list.append((i.title, i.score, i.url)) #recupera titolo, numero di upvote + il permalink


for post in post_list:
    for repost in reddit.subreddit('all').search('url:'+post[2]): #trova quando il post è stato citato
        subreddit_url = str(repost.subreddit)
        subreddit_url = "https://www.reddit.com/r/" + subreddit_url
        if subreddit_url in conspiracy_dict.keys():
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            conspiracy_dict[subreddit_url][1][0] +=1
        else:
            conspiracy_dict[subreddit_url]=[[],[1]]
            conspiracy_dict[subreddit_url][0].append("https://www.reddit.com"+repost.permalink)
            
                


df = pd.DataFrame(conspiracy_dict)
df.to_csv(r'results/conspiracy_data/conspiracy_top_url.csv',index=False)




# Creating the network
Now we need to create the network of subreddits involved.

In [ ]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [ ]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

network_list = list()

for subr in datasets:
        #print(f'now opening {subr}')
        try:
                df1 = pd.read_csv(subr)
        except:
                pass
        try:
                df1 = pd.read_csv(subr, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        sub_name=subr[subr.find("\\")+1:-4]
        for column in df1.columns:
                if column [:7] != "Unnamed":
                        col_name = column[column.find("/r/")+3:]
                        try:
                                network_list.append((sub_name, col_name, df1[column][1]))
                        except Exception as E:
                                print(E) 
                else: 
                        continue
print(network_list)

'''with open('Network.csv', 'w') as f:
      
    # using csv.writer method from CSV package
    write = csv.writer(f)
      
    write.writerow(["Source", "Target", "Weight"])
    write.writerows(network_list)'''

# Looking for subnetworks in the original csvs
Now that we have divided the main network in subnetworks we can llok for them in the original files.

In [6]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [8]:
import csv
import pandas as pd

datasets= [file for file in get_all_in_dir("results/1st_level")]
datasets.extend(file for file in get_all_in_dir("results/2nd_level"))

networks=[file for file in get_all_in_dir("results/subnetworks/mixed")]

for net in networks:
        try:
                df_network = pd.read_csv(net)
        except:
                pass
        try:
                df_network = pd.read_csv(net, encoding='utf8')
        except:
                print(f'unable to open {subr}')
                continue
        network_name=net[net.find("mixed\\")+6:-4]
        network_name=network_name+"_done"
        network_list=list()
        for subr in datasets:
                #print(f'now opening {subr}')
                try:
                        df_dataset = pd.read_csv(subr)
                except:
                        pass
                try:
                        df_dataset = pd.read_csv(subr, encoding='utf8')
                except:
                        print(f'unable to open {subr}')
                        continue
                sub_name=subr[subr.find("\\")+1:-4]
                for column in df_dataset.columns:
                        if column [:7] != "Unnamed":
                                col_name = column[column.find("/r/")+3:]
                                for index, row in df_network.iterrows():
                                        if (col_name == row["Source"] and sub_name == row["Target"]) or (col_name == row["Target"] and sub_name == row["Source"]):
                                                try:
                                                        network_list.append((sub_name, col_name, df_dataset[column][1],df_dataset[column][0]))
                                                except Exception as E:
                                                        print(E) 
                                        else: 
                                                continue
        
        
        try:
                with open(network_name+'.csv', 'w', encoding='utf-8') as f:
        # using csv.writer method from CSV package
                        write = csv.writer(f)
                        write.writerow(["Source", "Target", "Weight","Comments"])
                        write.writerows(network_list)
        except Exception as E:
                print(E)

                
                        

# Topic modeling

In [ ]:
import praw
from praw.models import MoreComments
import pandas as pd
reddit = praw.Reddit(client_id = "YUuwWt8-PMY44mNE7QJ0xw", client_secret= "TPohIg8dyysup5fmaQ96MhIQmunLOA", user_agent= "epds_son")

In [5]:
def get_all_in_dir(dir):
    for filename in os.listdir(dir):
        f = os.path.join(dir, filename)
        if os.path.isfile(f) and f[-4:] == ".csv":
            yield f

In [6]:
import random
def iterSample(iterable, samplesize):
    results = []
    for i, v in enumerate(iterable):
        r = random.randint(0, i)
        if r < samplesize:
            if i < samplesize:
                results.insert(r, v) # add first samplesize items in random order
            else:
                results[r] = v # at a decreasing rate, replace random items


    return results

Here we import the dataset and the comments from the posts in common between subreddits.

In [7]:
import pandas as pd
import re

to_scan= [file for file in get_all_in_dir("results/subnetworks/mixed")]
top_comments_list = dict()
for subreddit in to_scan:
        try: 
                df = pd.read_csv(subreddit, sep=',', encoding='utf8', on_bad_lines='skip').dropna()
                if os.path.isfile('comments_'+subreddit.split('\\')[1]):
                        start = len(pd.read_csv('comments_'+subreddit.split('\\')[1]))
                        df= df[(df.index == start).idxmin():]
        except:
                print('error',subreddit)
                continue
        for row in df.to_dict(orient='records'):
                try:
                        list_post=[]
                        list_post= row['Posts'][1:-1].split(',')
                except Exception as e:
                        continue
                result = ''
                num = 0
                for post_url in list_post:
                        if post_url == 0:
                                continue
                        post_url = post_url.replace(' ','')
                        #tmp =  re.sub("\[\]\'", "", post_url) NON FUNZIONA E NON CAPISCO PERCHé
                        try:
                                post = reddit.submission(url=post_url[1:-1])
                                try:
                                        #This allows up to 1 reply to each post.
                                        post.comments.replace_more(limit=1)
                                except Exception as e:
                                        print(e)
                                        print(f'{post}s comments was not accessible')
                                        continue
                                num += len(post.comments)
                                if num > 500:
                                        for comment in iterSample(post.comments, 500):
                                                try:
                                                        result += comment.body
                                                        for reply in comment.replies:
                                                                result+= reply.body
                                                        result=result.replace(',','')
                                                        result=result.replace('\n','\s')
                                                except:
                                                        continue
                                else:
                                        for comment in post.comments:
                                                try:
                                                        result += comment.body
                                                        for reply in comment.replies:
                                                                result+= reply.body
                                                        result=result.replace(',','')
                                                        result=result.replace('\n','\s')
                                                except:
                                                        continue
                        except Exception as e:
                                print(e)
                                print(post.permalink + ' is not accessible')
                                continue
                                        
                res=pd.DataFrame.from_dict([{'Source':row['Source'], 'Target':row['Target'],'NumberComments': num,'Comments':result}])
                if os.path.isfile('comments_'+subreddit.split('\\')[1]):
                        res.to_csv('comments_'+subreddit.split('\\')[1],  encoding='utf8', mode='a', header=False)
                else:
                        res.to_csv('comments_'+subreddit.split('\\')[1],  encoding='utf8')
                        
        print(subreddit, ' is finished')
                        
                        

C:\Users\Giorgia Sampo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


results/subnetworks/mixed\bitcoins_metareddit_news.csv  is finished


C:\Users\Giorgia Sampo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (2,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,2

results/subnetworks/mixed\left_right_bernie.csv  is finished


KeyboardInterrupt: 